In [ ]:
# 1) Drive bağla
from google.colab import drive
drive.mount('/content/drive')
print('✅ Drive bağlandı!')

In [ ]:
# 2) Yolları tanımla ve kontrol et
import os
from pathlib import Path

# Kaynak ve hedef
RAR_DIR = Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn_partial')
TARGET_DIR = Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn')

print('📁 RAR Klasörü:', RAR_DIR)
print('   Mevcut:', RAR_DIR.exists())
print()
print('📁 Hedef Klasör:', TARGET_DIR)
print('   Mevcut:', TARGET_DIR.exists())
print()

# RAR dosyalarını listele
if RAR_DIR.exists():
    print('📦 RAR Dosyaları:')
    rars = sorted(RAR_DIR.glob('*.rar'))
    total_size = 0
    for r in rars:
        size_gb = r.stat().st_size / (1024**3)
        total_size += size_gb
        print(f'   {r.name}: {size_gb:.2f} GB')
    print(f'\n   TOPLAM: {total_size:.2f} GB')
else:
    print('❌ RAR klasörü bulunamadı!')

In [ ]:
# 3) unrar kur
!apt-get update -qq && apt-get install -y unrar -qq
!unrar --version | head -1
print('✅ unrar kuruldu!')

In [ ]:
# 4) Hedef klasörleri oluştur
from pathlib import Path

TARGET_DIR = Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn')

folders = ['images', 'segmentation', 'instance', 'jsonlar']
for f in folders:
    (TARGET_DIR / f).mkdir(parents=True, exist_ok=True)
    print(f'✅ {TARGET_DIR / f}')

print('\n✅ Hedef klasörler hazır!')

In [ ]:
# 5) EXTRACTION - TÜM RAR'ları çıkar (detaylı log ile)
import os
import time
from pathlib import Path

RAR_DIR = Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn_partial')
TARGET_DIR = Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn')

# RAR -> Hedef klasör eşleştirmesi
extract_map = {
    'instance.rar': 'instance',
    'segmentation.rar': 'segmentation',
    'jsonlar.rar': 'jsonlar',
}
# images_01.rar ... images_10.rar -> images
for i in range(1, 11):
    extract_map[f'images_{i:02d}.rar'] = 'images'

def count_files_in_folder(folder):
    """Klasördeki dosya sayısını hızlıca say"""
    if not folder.exists():
        return 0
    try:
        return len([f for f in folder.rglob('*') if f.is_file()])
    except:
        return -1

print('='*60)
print('🚀 EXTRACTION BAŞLIYOR')
print('='*60)
print(f'Kaynak: {RAR_DIR}')
print(f'Hedef:  {TARGET_DIR}')
print('='*60)

# Extraction öncesi dosya sayıları
print('\n📊 EXTRACTION ÖNCESİ:')
before_counts = {}
for folder in ['images', 'segmentation', 'instance', 'jsonlar']:
    count = count_files_in_folder(TARGET_DIR / folder)
    before_counts[folder] = count
    print(f'   {folder}: {count:,} dosya')

total_start = time.time()
results = []  # Her RAR için sonuç kaydet

for rar_name, target_folder in extract_map.items():
    rar_path = RAR_DIR / rar_name
    target_path = TARGET_DIR / target_folder
    
    if not rar_path.exists():
        results.append({'rar': rar_name, 'status': 'NOT_FOUND', 'time': 0})
        print(f'\n⚠️  {rar_name} bulunamadı, atlanıyor...')
        continue
    
    # Extraction öncesi hedef klasör dosya sayısı
    before = count_files_in_folder(target_path)
    
    size_gb = rar_path.stat().st_size / (1024**3)
    print(f'\n{"─"*60}')
    print(f'📦 {rar_name} ({size_gb:.2f} GB)')
    print(f'   → {target_path}')
    print(f'   📊 Önceki dosya sayısı: {before:,}')
    print(f'   ⏳ Başlangıç: {time.strftime("%H:%M:%S")}')
    
    start = time.time()
    
    # unrar komutu: x = extract with full path, -o+ = overwrite, -idq = quiet
    cmd = f'unrar x -o+ -idq "{rar_path}" "{target_path}/"'
    rc = os.system(cmd)
    
    elapsed = time.time() - start
    
    # Extraction sonrası dosya sayısı
    after = count_files_in_folder(target_path)
    new_files = after - before
    
    if rc == 0:
        status = 'SUCCESS'
        print(f'   ✅ Tamamlandı! ({elapsed/60:.1f} dakika)')
        print(f'   📊 Sonraki dosya sayısı: {after:,} (+{new_files:,} yeni)')
    else:
        status = f'ERROR_{rc}'
        print(f'   ❌ HATA! Exit code: {rc}')
        print(f'   📊 Dosya sayısı: {after:,}')
    
    results.append({
        'rar': rar_name,
        'status': status,
        'time': elapsed,
        'before': before,
        'after': after,
        'new': new_files
    })

total_elapsed = time.time() - total_start

# Extraction sonrası dosya sayıları
print(f'\n{"="*60}')
print('📊 EXTRACTION SONRASI:')
print('─'*60)
for folder in ['images', 'segmentation', 'instance', 'jsonlar']:
    after = count_files_in_folder(TARGET_DIR / folder)
    before = before_counts[folder]
    diff = after - before
    print(f'   {folder:15} | Önce: {before:>10,} | Sonra: {after:>10,} | Yeni: +{diff:,}')

# Detaylı sonuç tablosu
print(f'\n{"="*60}')
print('📋 RAR BAZINDA SONUÇLAR:')
print('─'*60)
print(f'{"RAR":<20} {"Durum":<12} {"Süre":>8} {"Yeni Dosya":>12}')
print('─'*60)

success_count = 0
fail_count = 0
skip_count = 0
total_new = 0

for r in results:
    if r['status'] == 'SUCCESS':
        success_count += 1
        status_str = '✅ OK'
        new_str = f"+{r.get('new', 0):,}"
        total_new += r.get('new', 0)
    elif r['status'] == 'NOT_FOUND':
        skip_count += 1
        status_str = '⚠️ YOK'
        new_str = '-'
    else:
        fail_count += 1
        status_str = '❌ HATA'
        new_str = '?'
    
    time_str = f"{r['time']/60:.1f}dk" if r['time'] > 0 else '-'
    print(f'{r["rar"]:<20} {status_str:<12} {time_str:>8} {new_str:>12}')

print(f'\n{"="*60}')
print('📊 ÖZET')
print(f'{"="*60}')
print(f'✅ Başarılı: {success_count}')
print(f'❌ Başarısız: {fail_count}')
print(f'⚠️  Bulunamadı: {skip_count}')
print(f'⏱️  Toplam süre: {total_elapsed/60:.1f} dakika ({total_elapsed/3600:.2f} saat)')
print(f'{"="*60}')

In [ ]:
# 5.1) TEK RAR ÇIKAR - images_01.rar
# RAR_NUM değiştir: 1, 2, 3... 10
import subprocess
import time
from pathlib import Path

RAR_NUM = 1  # <-- DEĞİŞTİR: 1, 2, 3, 4, 5, 6, 7, 8, 9, 10

RAR_DIR = Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn_partial')
TARGET_DIR = Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn/images')

RAR_NAME = f'images_{RAR_NUM:02d}.rar'
rar_path = RAR_DIR / RAR_NAME

print('='*60)
print(f'📦 {RAR_NAME}')
print('='*60)

if not rar_path.exists():
    print(f'❌ HATA: {rar_path} bulunamadı!')
else:
    size_gb = rar_path.stat().st_size / (1024**3)
    print(f'📁 Kaynak: {rar_path}')
    print(f'   Boyut: {size_gb:.2f} GB')
    print(f'📁 Hedef: {TARGET_DIR}')
    
    # Hedef klasör oluştur
    TARGET_DIR.mkdir(parents=True, exist_ok=True)
    
    # Önceki dosya sayısı
    before = len(list(TARGET_DIR.glob('*.png'))) if TARGET_DIR.exists() else 0
    print(f'\n📊 Önceki dosya sayısı: {before:,}')
    print(f'⏳ Başlangıç: {time.strftime("%H:%M:%S")}')
    print('─'*60)
    
    start = time.time()
    
    # unrar çalıştır - hata mesajlarını göster
    result = subprocess.run(
        ['unrar', 'x', '-o+', str(rar_path), str(TARGET_DIR) + '/'],
        capture_output=True,
        text=True
    )
    
    elapsed = time.time() - start
    
    # Sonraki dosya sayısı
    after = len(list(TARGET_DIR.glob('*.png'))) if TARGET_DIR.exists() else 0
    new_files = after - before
    
    print('─'*60)
    print(f'⏱️  Süre: {elapsed/60:.1f} dakika')
    print(f'📊 Sonraki dosya sayısı: {after:,} (+{new_files:,} yeni)')
    
    if result.returncode == 0:
        print(f'\n✅ BAŞARILI!')
    else:
        print(f'\n❌ HATA! Return code: {result.returncode}')
        print(f'\n📋 Son çıktı:')
        print('\n'.join(result.stdout.split('\n')[-10:]))
        if result.stderr:
            print(f'\n⚠️ STDERR:')
            print(result.stderr)

print('='*60)

In [ ]:
# 6) DOĞRULAMA - Dosya sayıları
import os
from pathlib import Path

TARGET_DIR = Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn')

def count_files(folder, extensions=None):
    """Klasördeki dosyaları say"""
    if not folder.exists():
        return 0, 0
    
    count = 0
    size = 0
    for f in folder.rglob('*'):
        if f.is_file():
            if extensions is None or f.suffix.lower() in extensions:
                count += 1
                try:
                    size += f.stat().st_size
                except:
                    pass
    return count, size

print('='*60)
print('📊 DOSYA SAYILARI')
print('='*60)
print(f'Klasör: {TARGET_DIR}')
print('─'*60)

folders = [
    ('images', {'.png', '.jpg', '.jpeg'}),
    ('segmentation', {'.png'}),
    ('instance', {'.png'}),
    ('jsonlar', {'.json'}),
]

total_files = 0
total_size = 0

for folder_name, exts in folders:
    folder_path = TARGET_DIR / folder_name
    count, size = count_files(folder_path, exts)
    size_gb = size / (1024**3)
    total_files += count
    total_size += size
    
    status = '✅' if folder_path.exists() else '❌'
    print(f'{status} {folder_name:15} | {count:>10,} dosya | {size_gb:>8.2f} GB')

print('─'*60)
print(f'📦 {"TOPLAM":15} | {total_files:>10,} dosya | {total_size/(1024**3):>8.2f} GB')
print('='*60)

In [ ]:
# 7) Hızlı kontrol - İlk birkaç dosya
import os
from pathlib import Path

TARGET_DIR = Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn')

for folder in ['images', 'segmentation', 'instance', 'jsonlar']:
    folder_path = TARGET_DIR / folder
    print(f'\n📁 {folder}/')
    if folder_path.exists():
        files = list(folder_path.iterdir())[:5]
        for f in files:
            print(f'   {f.name}')
        if len(list(folder_path.iterdir())) > 5:
            print(f'   ... ve daha fazlası')
    else:
        print('   (klasör yok)')